In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pathlib
from glob import glob

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import pickle

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import *
from tensorflow.python.saved_model import signature_constants

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.exceptions import MlflowException

print(f'Pandas:{pd.__version__}, Numpy:{np.__version__}, Tensorflow:{tf.__version__}')

2023-01-26 11:54:39.617949: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-26 11:54:39.617985: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Pandas:1.3.4, Numpy:1.21.4, Tensorflow:2.7.0


In [2]:
def mlflow_setup():
    MLFLOW_TRACKING_URI ="sqlite:////home/pmspraju/tracking-server/mlflow.db" 
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

    return client

In [3]:
def create_mlflow_experiment(experiment_name):
    try:
        experiment_id = mlflow.create_experiment(
            experiment_name,
            #artifact_location=Path.cwd().joinpath("mlruns").as_uri(),
            artifact_location='//home/pmspraju/tracking-server/mlruns/',
            tags={"version": "v1", "priority": "P1"},
        )
    except Exception as MlflowException:
        print(f"Experiment exists")
        experiment= mlflow.set_experiment(experiment_name)
        # Examine the experiment details.
        print("Experiment_id: {}".format(experiment.experiment_id))
        print("Name: {}".format(experiment.name))
        print("Artifact Location: {}".format(experiment.artifact_location))
        print("Tags: {}".format(experiment.tags))
        print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
        print("Last Updated timestamp: {}".format(experiment.last_update_time))

In [4]:
# Resize an image based on a scale factor. Take in to consideration that it receives an image represented as Numpy array.
def resize_image(image_array, factor):
    original_image = Image.fromarray(image_array)
    new_size = np.array(original_image.size) * factor
    new_size = new_size.astype(np.int32)
    new_size = tuple(new_size)

    resized = original_image.resize(new_size)
    resized = img_to_array(resized)
    resized = resized.astype(np.uint8)

    return resized

In [5]:
# Tightly crop an image. We need the image to fit nicely when we apply a sliding window to extract patches later. 
# SCALE is the actor we want the network to learn how to enlarge images by
def tight_crop_image(image, scale):
    height, width = image.shape[:2]
    width -= int(width % scale)
    height -= int(height % scale)

    return image[:height, :width]

In [6]:
# Reduce resolution of an imae by downsizing and then upsizing 
def downsize_upsize_image(image, scale):
    scaled = resize_image(image, 1.0 / scale)
    scaled = resize_image(scaled, scale / 1.0)

    return scaled

In [7]:
# Crop patches from input images. input_dim is the height&width of the images that is input to the network
def crop_input(image, x, y, input_dim):
    x_slice = slice(x, x + input_dim)
    y_slice = slice(y, y + input_dim)

    return image[y_slice, x_slice]

In [8]:
# Crop patches of output images. label_size is the height&width of the images output by the network. 
# pad is the number of pixels used as padding to ensure we are cropping the roi accurately
def crop_output(image, x, y, label_size, pad):
    y_slice = slice(y + pad, y + pad + label_size)
    x_slice = slice(x + pad, x + pad + label_size)
    #print(x, y , x_slice, y_slice)

    return image[y_slice, x_slice]


In [9]:
# method to prepare data
def prepare_data(path):
    SEED = 999
    SUBSET_SIZE = 100 #1500
    SCALE = 2.0
    INPUT_DIM = 33
    LABEL_SIZE = 21
    PAD = int((INPUT_DIM - LABEL_SIZE) / SCALE)
    STRIDE = 14
    
    np.random.seed(SEED)
    pattern = (path / 'images' / '*.png')
    file_patterns = str(pattern)
    dataset_paths = [*glob(file_patterns)]
    
    dataset_paths = np.random.choice(dataset_paths, SUBSET_SIZE)

    data = []; labels = []; cnt = 0

    for image_path in dataset_paths:
        image = load_img(image_path)
        image = img_to_array(image)
        image = image.astype(np.uint8); #Image.fromarray(image).show()
        image = tight_crop_image(image, SCALE); #Image.fromarray(image).show()
        scaled = downsize_upsize_image(image, SCALE); #Image.fromarray(scaled).show()

        height, width = image.shape[:2]
        #print(f'height:{height},width:{width}')
        
        for y in range(0, height - INPUT_DIM + 1, STRIDE):
            for x in range(0, width - INPUT_DIM + 1, STRIDE):
                crop = crop_input(scaled, x, y, INPUT_DIM); #Image.fromarray(crop).show()
                target = crop_output(image, x, y, LABEL_SIZE, PAD);#Image.fromarray(target).show()
                
                #cnt = cnt + 1
                
                data.append(crop)#data.append(np.array(crop).flatten()) #use np.reshape(fi,(33,33,3)) to read        
                labels.append(target)#labels.append(np.array(target).flatten());#use np.reshape(fi,(33,33,3)) to read  

                #fname = f'train/images/image_{y}_{x}.png' 
                #Image.fromarray(crop).save(os.path.join(path,fname))   

                #fname = f'train/labels/label_{y}_{x}.png'              
                #Image.fromarray(target).save(os.path.join(path,fname)) 
                #break
            #break
        #break
    #print(cnt)    
    return [data, labels]
    #pd.DataFrame({'image': data, 'label': labels})


In [11]:
# # Test data
# path = pathlib.Path('/mnt/c/Users/pmspr/Documents/Machine Learning/Courses/Tensorflow Cert/Data/dogscats')
# [data, labels] = prepare_data(path)
# print(f'shape:{np.array(labels).shape}')
# print(f'shape:{np.array(data).shape}')
# labels = tf.convert_to_tensor(labels, np.int32)
# print(labels.shape)

In [12]:
def build_model(height, width, depth):
    input = Input(shape=(height, width, depth))

    x = Conv2D(filters=64, 
               kernel_size=(9,9),
               kernel_initializer='he_normal'
               ) (input)
    
    x = ReLU()(x)

    x = Conv2D(filters=32, 
               kernel_size=(1,1),
               kernel_initializer='he_normal'
                ) (x)

    x = ReLU()(x)

    output = Conv2D(filters=depth, 
                    kernel_size=(5,5),
                    kernel_initializer='he_normal'
                    ) (x)

    return Model(input, output)


In [14]:
def train_model(dim):
    path = pathlib.Path('/mnt/c/Users/pmspr/Documents/Machine Learning/Courses/Tensorflow Cert/Data/dogscats')
    [data, labels] = prepare_data(path)
    data = tf.convert_to_tensor(data, np.int32)
    labels = tf.convert_to_tensor(labels, np.int32)
    print(f'Data shape:{data.shape}, Labels shape"{labels.shape}')

    EPOCHS = 1 #12
    optimizer = Adam(learning_rate=1e-3, decay=1e-3 / EPOCHS)
    model = build_model(dim, dim, 3)
    model.compile(loss='mse', optimizer=optimizer)
    BATCH_SIZE = 64
    
    model.fit(data, labels, batch_size=BATCH_SIZE, epochs = EPOCHS)

    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)

    tag=[tf.compat.v1.saved_model.tag_constants.SERVING]
    key=signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
    saved_model_path = os.path.join(path, )

    mlflow.tensorflow.log_model(model=model,
                                artifact_path="tf-models")

    # write model summary
    summary = []
    model.summary(print_fn=summary.append)
    summary = "\n".join(summary)
    summary_path = os.path.join(path, "model_summary.txt")
    with open(summary_path, "w") as f:
        f.write(summary)
    mlflow.log_artifact(summary_path)
    os.remove(summary_path)

    # write model as json file
    model_json_path = os.path.join(path, "model.json")
    with open(model_json_path, "w") as f:
        f.write(model.to_json())
    mlflow.log_artifact(model_json_path)
    os.remove(model_json_path)

In [15]:
def main():
    experiment_name = 'IMAGE-RESOLUTION'
    dim = 33
    client = mlflow_setup()
    create_mlflow_experiment(experiment_name)

    with mlflow.start_run() as run:
        print("MLflow:")
        print("  run_id:",run.info.run_id)
        print("  experiment_id:",run.info.experiment_id)
        mlflow.set_tag("version.mlflow", mlflow.__version__)
        mlflow.set_tag("version.tensorflow", tf.__version__)
        train_model(dim)

In [17]:
if __name__ == "__main__":
    main()

Experiment exists
Experiment_id: 3
Name: IMAGE-RESOLUTION
Artifact Location: //home/pmspraju/tracking-server/mlruns/
Tags: {'version': 'v1', 'priority': 'P1'}
Lifecycle_stage: active
Last Updated timestamp: 1674755884516
MLflow:
  run_id: 7f80f63bb25846f9bdf391a920727660
  experiment_id: 3
Data shape:(67477, 33, 33, 3), Labels shape"(67477, 21, 21, 3)
1055/1055 [==============================] - 67s 63ms/step - loss: 222.2619


2023/01/26 12:01:42 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2023-01-26 12:01:42.255382: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpeto0xin3/model/data/model/assets


/home/pmspraju/.local/share/virtualenvs/imageResolution-c2hlZ-SW/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [75]:
!jupyter nbconvert --to script imageResolution.ipynb

[NbConvertApp] Converting notebook imageResolution.ipynb to script
[NbConvertApp] Writing 6499 bytes to imageResolution.py


In [ ]:
path = pathlib.Path('/mnt/c/Users/pmspr/Documents/Machine Learning/Courses/Tensorflow Cert/Data/dogscats')
    # df = prepare_data(path)
    # with open(os.path.join(path,'train.txt'), 'wb') as fp:
    #     pickle.dump(df[0], fp, pickle.HIGHEST_PROTOCOL)
    #np.savetxt(os.path.join(path,'train.txt'), df.values, fmt='%d')
    # with open(os.path.join(path,'train.txt'), 'a') as f:
    #     dfAsString = df.to_string(header=False, index=False)
    #     f.write(dfAsString)